<a href="https://colab.research.google.com/github/johntanas/it1244project/blob/main/code/model_one_hot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

In [157]:
path_to_evm_label="/content/expo_moving_k2_labels.csv"
path_to_app="/content/cleaned_application.csv"
cleaned_app_df=pd.read_csv(path_to_app,index_col=0)
ewm_df_status=pd.read_csv(path_to_evm_label,index_col=0)

In [4]:
labels_to_int_evm={"bad":0,"good":1}

In [5]:
ewm_df=ewm_df_status.drop(columns="status").replace({"label":labels_to_int_evm})

In [6]:
dep_var="label"
random_state=42

In [7]:
from sklearn.model_selection import ShuffleSplit
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier,export_text,plot_treefrom fastai.tabular.all import *
from sklearn.metrics import classification_report, confusion_matrix,ConfusionMatrixDisplay

In [73]:
df1=ewm_df.merge(cleaned_app_df,how="inner",on="id").drop("id",axis=1)

In [139]:
df=df1.replace(to_replace = ("Y", "N"),
             value = (1, 0))

In [140]:
gender = pd.get_dummies(df["gender"])
income_type = pd.get_dummies(df["income_type"])
education_level = pd.get_dummies(df["education_level"])
family_status = pd.get_dummies(df["family_status"])
house_type = pd.get_dummies(df["house_type"])
job = pd.get_dummies(df["job"])
one_hot=df.drop(columns = ["gender", "income_type", "education_level",
                   "family_status", "house_type", "job"])
one_hot = pd.concat([one_hot, gender, income_type, education_level, family_status,
                house_type, job], axis = 1)
yvals = pd.DataFrame(one_hot[dep_var])
xvals = one_hot.drop(columns = dep_var)
from sklearn.model_selection import train_test_split
xtrain, xtest, y, valid_y = train_test_split(xvals, yvals, test_size = 0.20,
                                                random_state = random_state)

# Standardizing the training and test data
from sklearn.preprocessing import StandardScaler
std_x = StandardScaler()
std_x.fit(xtrain)
xs = std_x.transform(xtrain)
valid_xs = std_x.transform(xtest)

In [156]:
pd.DataFrame(xs)

,0,1,2,3,4,5,6,7,8,9,...,42,43,44,45,46,47,48,49,50,51
0,1.322403,-1.416144,-0.562311,0.253557,0.847678,-0.557377,1.881126,-0.628469,-0.299495,-0.184840,...,-0.080549,-0.318122,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
1,-0.756199,-1.416144,0.769710,2.175105,0.155277,0.492201,-0.531597,-0.628469,-0.299495,0.891466,...,-0.080549,3.143453,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
2,-0.756199,0.706143,-0.562311,-0.590049,1.100899,0.677858,-0.531597,-0.628469,-0.299495,-0.184840,...,-0.080549,-0.318122,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
3,1.322403,0.706143,-0.562311,0.394158,0.142709,0.205053,-0.531597,-0.628469,3.338951,-0.184840,...,-0.080549,-0.318122,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
4,-0.756199,0.706143,-0.562311,1.893903,-1.639846,1.206360,-0.531597,-0.628469,-0.299495,-0.184840,...,-0.080549,-0.318122,-0.181527,2.162272,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6045,-0.756199,0.706143,-0.562311,3.299913,1.210985,0.366368,-0.531597,-0.628469,-0.299495,-0.184840,...,-0.080549,-0.318122,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
6046,-0.756199,-1.416144,-0.562311,0.019222,0.321220,0.262812,-0.531597,-0.628469,-0.299495,-1.261146,...,-0.080549,-0.318122,-0.181527,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
6047,-0.756199,0.706143,-0.562311,1.097163,-1.354042,0.792140,-0.531597,-0.628469,-0.299495,-0.184840,...,-0.080549,-0.318122,-0.181527,2.162272,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775
6048,-0.756199,-1.416144,-0.562311,-1.058719,1.714634,0.565226,1.881126,-0.628469,-0.299495,-1.261146,...,-0.080549,-0.318122,5.508825,-0.462476,-0.08753,-0.046405,-0.394007,-0.068188,-0.141039,-0.061775


In [141]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [142]:
int_to_labels_evm={"bad":1,"good":2}

https://www.kaggle.com/code/umerkk12/credit-card-predictive-analysis#there-are-9516-rows.

In [143]:
classifiers = {
    "LogisticRegression" : LogisticRegression(),
    "KNeighbors" : KNeighborsClassifier(),
    "SVC" : SVC(),
    "DecisionTree" : DecisionTreeClassifier(),
    "RandomForest" : RandomForestClassifier(),
    "XGBoost" : XGBClassifier()
}

In [144]:
for key, classifier in classifiers.items():
    classifier.fit(xs, y)
    prediction = classifier.predict(valid_xs)
    print("Model "+key+ " using unbalanced data")
    print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Model LogisticRegression using unbalanced data
              precision    recall  f1-score   support

         bad       0.67      0.00      0.01       410
        good       0.73      1.00      0.84      1103

    accuracy                           0.73      1513
   macro avg       0.70      0.50      0.43      1513
weighted avg       0.71      0.73      0.62      1513

Model KNeighbors using unbalanced data
              precision    recall  f1-score   support

         bad       0.27      0.12      0.16       410
        good       0.73      0.88      0.80      1103

    accuracy                           0.67      1513
   macro avg       0.50      0.50      0.48      1513
weighted avg       0.60      0.67      0.63      1513



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model SVC using unbalanced data
              precision    recall  f1-score   support

         bad       0.50      0.00      0.00       410
        good       0.73      1.00      0.84      1103

    accuracy                           0.73      1513
   macro avg       0.61      0.50      0.42      1513
weighted avg       0.67      0.73      0.62      1513

Model DecisionTree using unbalanced data
              precision    recall  f1-score   support

         bad       0.27      0.30      0.28       410
        good       0.73      0.69      0.71      1103

    accuracy                           0.58      1513
   macro avg       0.50      0.50      0.50      1513
weighted avg       0.60      0.58      0.59      1513



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Model RandomForest using unbalanced data
              precision    recall  f1-score   support

         bad       0.30      0.06      0.10       410
        good       0.73      0.95      0.83      1103

    accuracy                           0.71      1513
   macro avg       0.52      0.50      0.46      1513
weighted avg       0.61      0.71      0.63      1513



/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model XGBoost using unbalanced data
              precision    recall  f1-score   support

         bad       0.43      0.01      0.01       410
        good       0.73      1.00      0.84      1103

    accuracy                           0.73      1513
   macro avg       0.58      0.50      0.43      1513
weighted avg       0.65      0.73      0.62      1513



In [146]:
import keras
import tensorflow as tf
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten


In [147]:
one_hot_y=pd.get_dummies(y["label"])
one_hot_valid_y=pd.get_dummies(valid_y["label"])

In [148]:
model=keras.Sequential([
keras.Input(shape=(52,)),
keras.layers.Dense(1024),
keras.layers.Activation("sigmoid"),
keras.layers.Dense(512),
keras.layers.Activation(activation="relu"),
keras.layers.Dense(2),
keras.layers.Activation(activation="softmax")])

In [149]:
opt = tf.optimizers.SGD(learning_rate=0.01)
model.compile(loss=tf.keras.losses.BinaryFocalCrossentropy(), optimizer=opt, metrics=['accuracy',tf.keras.metrics.Recall()])
epochs = 10
history=model.fit(xs, one_hot_y,batch_size=32, epochs=epochs,validation_data=(valid_xs,one_hot_valid_y))

Epoch 1/10
190/190 [==============================] - 3s 12ms/step - loss: 0.1525 - accuracy: 0.7226 - recall_12: 0.7226 - val_loss: 0.1858 - val_accuracy: 0.7290 - val_recall_12: 0.7290
Epoch 2/10
190/190 [==============================] - 2s 13ms/step - loss: 0.1512 - accuracy: 0.7250 - recall_12: 0.7250 - val_loss: 0.2023 - val_accuracy: 0.7290 - val_recall_12: 0.7290
Epoch 3/10
190/190 [==============================] - 3s 14ms/step - loss: 0.1509 - accuracy: 0.7220 - recall_12: 0.7220 - val_loss: 0.1714 - val_accuracy: 0.7290 - val_recall_12: 0.7290
Epoch 4/10
190/190 [==============================] - 3s 17ms/step - loss: 0.1504 - accuracy: 0.7246 - recall_12: 0.7246 - val_loss: 0.1788 - val_accuracy: 0.3510 - val_recall_12: 0.3510
Epoch 5/10
190/190 [==============================] - 3s 15ms/step - loss: 0.1499 - accuracy: 0.7240 - recall_12: 0.7240 - val_loss: 0.1586 - val_accuracy: 0.7257 - val_recall_12: 0.7257
Epoch 6/10
190/190 [==============================] - 2s 10ms/ste

In [119]:
y_prob=model.predict(valid_xs)
prediction = y_prob.argmax(axis=-1)
print("Model Neural Net")
print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

48/48 [==============================] - 0s 3ms/step
Model Neural Net
              precision    recall  f1-score   support

         bad       0.00      0.00      0.00       410
        good       0.73      1.00      0.84      1103

    accuracy                           0.73      1513
   macro avg       0.36      0.50      0.42      1513
weighted avg       0.53      0.73      0.61      1513



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [120]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_balanced, y_balanced = oversample.fit_resample(xs, y)

In [121]:
for key, classifier in classifiers.items():
    classifier.fit(X_balanced, y_balanced)
    prediction = classifier.predict(valid_xs)
    print("Model "+key)
    print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/neighbors/_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Model LogisticRegression
              precision    recall  f1-score   support

         bad       0.27      0.50      0.35       410
        good       0.73      0.50      0.60      1103

    accuracy                           0.50      1513
   macro avg       0.50      0.50      0.47      1513
weighted avg       0.61      0.50      0.53      1513

Model KNeighbors
              precision    recall  f1-score   support

         bad       0.29      0.45      0.35       410
        good       0.74      0.60      0.66      1103

    accuracy                           0.56      1513
   macro avg       0.52      0.52      0.51      1513
weighted avg       0.62      0.56      0.58      1513



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model SVC
              precision    recall  f1-score   support

         bad       0.27      0.46      0.34       410
        good       0.73      0.55      0.63      1103

    accuracy                           0.52      1513
   macro avg       0.50      0.50      0.49      1513
weighted avg       0.61      0.52      0.55      1513

Model DecisionTree
              precision    recall  f1-score   support

         bad       0.29      0.35      0.32       410
        good       0.74      0.68      0.71      1103

    accuracy                           0.59      1513
   macro avg       0.51      0.51      0.51      1513
weighted avg       0.62      0.59      0.60      1513



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Model RandomForest
              precision    recall  f1-score   support

         bad       0.30      0.18      0.22       410
        good       0.74      0.85      0.79      1103

    accuracy                           0.67      1513
   macro avg       0.52      0.51      0.51      1513
weighted avg       0.62      0.67      0.64      1513



/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Model XGBoost
              precision    recall  f1-score   support

         bad       0.29      0.10      0.15       410
        good       0.73      0.91      0.81      1103

    accuracy                           0.69      1513
   macro avg       0.51      0.50      0.48      1513
weighted avg       0.61      0.69      0.63      1513



In [129]:
one_hot_y_balanced=pd.get_dummies(y_balanced["label"])

In [127]:
model2=keras.Sequential([
keras.Input(shape=(49,)),
keras.layers.Dense(1024),
keras.layers.Activation("sigmoid"),
keras.layers.Dense(512),
keras.layers.Activation(activation="relu"),
keras.layers.Dense(2),
keras.layers.Activation(activation="softmax")])

In [132]:
opt = tf.optimizers.SGD(learning_rate=0.01)
model2.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=opt, metrics=['accuracy',tf.keras.metrics.Recall()])
epochs = 10
history=model2.fit(X_balanced,one_hot_y_balanced,batch_size=32, epochs=epochs,validation_data=(valid_xs,one_hot_valid_y))

Epoch 1/10
275/275 [==============================] - 5s 14ms/step - loss: 0.6924 - accuracy: 0.5214 - recall_11: 0.5214 - val_loss: 0.7079 - val_accuracy: 0.3496 - val_recall_11: 0.3496
Epoch 2/10
275/275 [==============================] - 3s 12ms/step - loss: 0.6923 - accuracy: 0.5138 - recall_11: 0.5138 - val_loss: 0.6922 - val_accuracy: 0.4983 - val_recall_11: 0.4983
Epoch 3/10
275/275 [==============================] - 3s 12ms/step - loss: 0.6913 - accuracy: 0.5275 - recall_11: 0.5275 - val_loss: 0.6614 - val_accuracy: 0.7118 - val_recall_11: 0.7118
Epoch 4/10
275/275 [==============================] - 4s 14ms/step - loss: 0.6904 - accuracy: 0.5378 - recall_11: 0.5378 - val_loss: 0.6911 - val_accuracy: 0.5063 - val_recall_11: 0.5063
Epoch 5/10
275/275 [==============================] - 3s 11ms/step - loss: 0.6908 - accuracy: 0.5269 - recall_11: 0.5269 - val_loss: 0.7251 - val_accuracy: 0.3139 - val_recall_11: 0.3139
Epoch 6/10
275/275 [==============================] - 3s 11ms/ste

In [133]:
y_prob=model2.predict(valid_xs)
prediction = y_prob.argmax(axis=-1)
print("Model Neural Net")
print(classification_report(valid_y, prediction,target_names=int_to_labels_evm))

48/48 [==============================] - 0s 3ms/step
Model Neural Net
              precision    recall  f1-score   support

         bad       0.27      0.52      0.36       410
        good       0.73      0.48      0.58      1103

    accuracy                           0.49      1513
   macro avg       0.50      0.50      0.47      1513
weighted avg       0.61      0.49      0.52      1513

